In [1]:
import yt
from yt import derived_field

import numpy as np 

from astropy.table import Table
import astropy.units as u
from matplotlib.colors import LogNorm

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
import matplotlib as mpl
mpl.rcParams['font.family'] = 'stixgeneral'
import matplotlib.pyplot as plt
import matplotlib.lines as lines

import cmasher as cmr

import os

import h5py

from foggie.utils.foggie_load import *
from foggie.clumps.clump_finder import *


In [2]:
#Load in the snapshot
code_dir = '/Users/ctrapp/Documents/GitHub/foggie/'
data_dir = '/Volumes/FoggieCam/foggie_halos/'

gal_id = "008508" ## Tempest
snapnum = "DD0967"

if gal_id=="008508":
    gal_name = "Tempest"
if gal_id=="005036":
    gal_name = "Maelstrom"
if gal_id == "002392":
    gal_name = "Hurricane"
if gal_id=="005016":
    gal_name = "Squall"
if gal_id=="002392":
    gal_name = "Hurricane"
if gal_id=="002878":
    gal_name = "Cyclone"
    
GalName=gal_name

gal_name+="_"+snapnum

#refinement = "nref11n"
refinement = "nref11c_nref9f"
run=refinement
snap_name = data_dir + "halo_"+gal_id+"/"+refinement+"/"+snapnum+"/"+snapnum
#snap_name = data_dir + "halo_"+gal_id+"/natural/"+snapnum+"/"+snapnum

trackname = code_dir+"/foggie/halo_tracks/"+gal_id+"/nref11n_selfshield_15/halo_track_200kpc_nref9"

if refinement=="natural" or refinement=="nref11n":
    gal_name+="_natural"

halo_c_v_name = code_dir+"/foggie/halo_infos/"+gal_id+"/"+refinement+"/halo_c_v"

#particle_type_for_angmom = 'young_stars' ##Currently the default
particle_type_for_angmom = 'gas' #Should be defined by gas with Temps below 1e4 K

catalog_dir = code_dir + 'foggie/halo_infos/' + gal_id + '/'+refinement+'/'
#smooth_AM_name = catalog_dir + 'AM_direction_smoothed'
smooth_AM_name = None


ds, refine_box = foggie_load(snap_name, trackfile_name=trackname, halo_c_v_name=halo_c_v_name, do_filter_particles=True,disk_relative=True,particle_type_for_angmom=particle_type_for_angmom,smooth_AM_name = smooth_AM_name)

yt : [INFO     ] 2025-10-02 14:14:55,529 Parameters: current_time              = 274.56652781829
yt : [INFO     ] 2025-10-02 14:14:55,530 Parameters: domain_dimensions         = [256 256 256]
yt : [INFO     ] 2025-10-02 14:14:55,530 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-10-02 14:14:55,530 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-10-02 14:14:55,530 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2025-10-02 14:14:55,530 Parameters: current_redshift          = 1.0043884467376
yt : [INFO     ] 2025-10-02 14:14:55,531 Parameters: omega_lambda              = 0.715
yt : [INFO     ] 2025-10-02 14:14:55,531 Parameters: omega_matter              = 0.285
yt : [INFO     ] 2025-10-02 14:14:55,531 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-02 14:14:55,531 Parameters: hubble_constant           = 0.695


Opening snapshot /Volumes/FoggieCam/foggie_halos/halo_008508/nref11c_nref9f/DD0967/DD0967


Parsing Hierarchy : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5908/5908 [00:00<00:00, 33367.78it/s]
yt : [INFO     ] 2025-10-02 14:14:55,762 Gathering a field list (this may take a moment.)
yt : [WARNING  ] 2025-10-02 14:14:56,080 Field ('index', 'cell_id') already exists. To override use `force_override=True`.


get_refine_box: using this location:      col1       col2     col3     col4     col5     col6     col7   col8
------------- -------- -------- -------- -------- -------- -------- ----
1.00438844674 0.491497 0.481944 0.503607 0.493497 0.483944 0.505607    9
Will look for halo_c_v_file:  /Users/ctrapp/Documents/GitHub/foggie//foggie/halo_infos/008508/nref11c_nref9f/halo_c_v
Found halo_c_v file: /Users/ctrapp/Documents/GitHub/foggie//foggie/halo_infos/008508/nref11c_nref9f/halo_c_v
Using halo_c_v catalog file:  /Users/ctrapp/Documents/GitHub/foggie//foggie/halo_infos/008508/nref11c_nref9f/halo_c_v  for center style  catalog
Pulling halo center from catalog file
halo center in kpc:  [35353.348781   34666.28976357 36223.74219945] kpc
halo velocity in km/s:  [ -28.43609939 -146.54126284   59.01444901] km/s
made it to here!
ds.halo_velocity_kms =  [ -28.43609939 -146.54126284   59.01444901] km/s  so we can add the centered velocity and energy fields
filtering young_stars particles...
filtering

In [3]:
#Load in the clumps, add relevant fields for tracking
from foggie.clumps.clump_finder.utils_clump_finder import add_cell_id_field
from foggie.clumps.clump_finder.utils_clump_finder import add_leaf_id_field

clump_dir = '/Users/ctrapp/Documents/foggie_analysis/clump_project/clump_catalog/'
hierarchy_file = clump_dir + GalName+"_"+snapnum+"_"+run+"_ClumpTree.h5"

add_cell_id_field(ds)
add_leaf_id_field(ds,hierarchy_file)

/Users/ctrapp/Documents/GitHub/foggie/foggie/clumps/clump_finder/utils_clump_finder.py:202: RuntimeWarning: invalid value encountered in cast
  u_id = np.round(gids + np.multiply(c_ids , max_gid+1)).astype(np.uint64)


In [ ]:
### ~~~Vida Modify Here~~~ ###

#Function to calculate and display bulk clump stats
import trident
trident.add_ion_fields(ds, ions=['O VI','Mg II'])


def load_clump_stats(ds,source_cut,verbose=False):
    leaf_ids = source_cut['gas','leaf_id']
    gas_mass = source_cut['gas','cell_mass']
    cell_volume = source_cut['gas','cell_volume']
    radial_velocity = source_cut['gas','radial_velocity_corrected']
    
    h_p0_number_density = source_cut['gas','H_p0_number_density']
    o_p6_number_density = source_cut['gas','O_p5_number_density']
    mg_p2_number_density = source_cut['gas','Mg_p1_number_density']

    clump_mass = np.zeros((int(np.max(leaf_ids.v+1))))
    clump_volume = np.copy(clump_mass)
    clump_radius = np.copy(clump_mass)
    clump_rvel  = np.copy(clump_mass)
    clump_h_p0  = np.copy(clump_mass)
    clump_o_p6  = np.copy(clump_mass)
    clump_mg_p2 = np.copy(clump_mass)
    
    for leaf_id in np.unique(leaf_ids):
        if leaf_id<0: continue
        mask = (leaf_ids==leaf_id)
        clump_mass[int(leaf_id)] = np.sum(gas_mass[mask].in_units('Msun'))
        clump_volume[int(leaf_id)] = np.sum(cell_volume[mask].in_units('kpc**3'))
        clump_radius[int(leaf_id)] = np.power(clump_volume[int(leaf_id)],1./3.)
        #Mass Weighted
        clump_rvel[int(leaf_id)] = np.sum(np.multiply(gas_mass[mask],radial_velocity[mask].in_units('km/s'))) / np.sum(gas_mass[mask])
        

        ### ~~~Vida Modify Here~~~ ###
        
        #Volume Weighted
        weight = cell_volume[mask]
        norm = np.sum(weight)
        
        clump_h_p0[int(leaf_id)]  = np.sum(np.multiply(weight, h_p0_number_density[mask].in_units("cm**-3"))) / norm
        clump_o_p6[int(leaf_id)]  = np.sum(np.multiply(weight, o_p6_number_density[mask].in_units("cm**-3"))) / norm
        clump_mg_p2[int(leaf_id)] = np.sum(np.multiply(weight, mg_p2_number_density[mask].in_units("cm**-3"))) / norm
        
        ### ~~~                ~~~ ###

        if verbose:
            print("For Leaf",int(leaf_id))
            print(f"    Clump Mass      = {clump_mass[int(leaf_id)]:.3e} Msun")
            print(f"    Clump Volume    = {clump_volume[int(leaf_id)]:.3e} kpc^3")
            print(f"    Clump ~Radius   = {clump_radius[int(leaf_id)]:.3e} kpc")
            print(f"    Rad. Vel.       = {clump_rvel[int(leaf_id)]:.3f} km/s")
            print(f"    HI Num. Dens.   = {clump_h_p0[int(leaf_id)]:.3e} cm^-3")
            print(f"    OVI Num. Dens.  = {clump_o_p6[int(leaf_id)]:.3e} cm^-3")
            print(f"    MgII Num. Dens. = {clump_mg_p2[int(leaf_id)]:.3e} cm^-3")

            #HVC and IVC descriptions are stupid and pointless from a theoretical point of view
            #HVC Doppler vel > 90
            #IVC: 90 > Doppler vel > 60

    return clump_mass,clump_radius,clump_rvel,clump_h_p0,clump_o_p6,clump_mg_p2,leaf_ids




clump_mass,clump_radius,clump_rvel,clump_h_p0,clump_o_p6,clump_mg_p2,leaf_ids = load_clump_stats(ds,refine_box,verbose=True)


yt : [WARNING  ] 2025-10-02 14:15:04,149 Field ('index', 'cell_id_2') was added without specifying units or dimensions, auto setting units to 'dimensionless'
yt : [WARNING  ] 2025-10-02 14:17:14,484 Field ('gas', 'leaf_id') was added without specifying units or dimensions, auto setting units to 'dimensionless'
/opt/anaconda3/envs/foggie/lib/python3.12/site-packages/unyt/array.py:1972: RuntimeWarning: invalid value encountered in divide
  out_arr = func(


For Leaf 54
    Clump Mass      = 1.174e+07 Msun
    Clump Volume    = 1.283e-01 kpc^3
    Clump ~Radius   = 5.044e-01 kpc
    Rad. Vel.       = -75.609 km/s
    HI Num. Dens.   = 2.804e+00 cm^-3
    OVI Num. Dens.  = 0.000e+00 cm^-3
    MgII Num. Dens. = 8.832e-06 cm^-3
For Leaf 67
    Clump Mass      = 7.000e+03 Msun
    Clump Volume    = 3.384e+01 kpc^3
    Clump ~Radius   = 3.235e+00 kpc
    Rad. Vel.       = 394.100 km/s
    HI Num. Dens.   = 4.917e-13 cm^-3
    OVI Num. Dens.  = 5.311e-13 cm^-3
    MgII Num. Dens. = 0.000e+00 cm^-3
For Leaf 69
    Clump Mass      = 1.563e+05 Msun
    Clump Volume    = 1.889e+02 kpc^3
    Clump ~Radius   = 5.738e+00 kpc
    Rad. Vel.       = 69.742 km/s
    HI Num. Dens.   = 2.949e-11 cm^-3
    OVI Num. Dens.  = 1.178e-10 cm^-3
    MgII Num. Dens. = 0.000e+00 cm^-3
For Leaf 72
    Clump Mass      = 3.056e+04 Msun
    Clump Volume    = 4.928e+00 kpc^3
    Clump ~Radius   = 1.702e+00 kpc
    Rad. Vel.       = 37.690 km/s
    HI Num. Dens.   = 7.663e

In [ ]:
#Identify which clumps are in the disk so they can be ignored

from foggie.clumps.clump_finder import GetClumpsInDisk

disk_catalog_dir = '/Users/ctrapp/Documents/foggie_analysis/disk_project/disk_catalog/'
disk_file = disk_catalog_dir + GalName+"_"+snapnum+"_"+run+"_Disk.h5"

disk_clump_ids = GetClumpsInDisk(np.unique(leaf_ids.v[leaf_ids>=0]).astype(int), hierarchy_file, disk_file)

In [ ]:
### ~~~Vida Modify Here~~~ ###

#Define clumps based on some criteria, save all clump ids that fit this criteria

def print_clump_stats(mass,radius,rvel,vdoppler,h_p0,o_p6,mg_p2):
            print(f"    Clump Mass      = {mass:.3e} Msun")
            print(f"    Clump ~Radius   = {radius:.3e} kpc")
            print(f"    Rad. Vel.       = {rvel:.3e} km/s")
            print(f"    HI Num. Dens.   = {h_p0:.3e} cm^-3")
            print(f"    OVI Num. Dens.  = {o_p6:.3e} cm^-3")
            print(f"    MgII Num. Dens. = {mg_p2:.3e} cm^-3")


### ~~~Vida Modify Here~~~ ###

o_p6_cutoff = 1e-10
mg_p2_cutoff = 1e-10
ovi_clump_ids = np.where((clump_o_p6 > o_p6_cutoff))[0]
mgii_clump_ids = np.where((clump_mg_p2 > mg_p2_cutoff))[0]

### ~~~                ~~~ ###


#Filter out the disk
ovi_clump_ids = ovi_clump_ids[~np.isin(ovi_clump_ids, disk_clump_ids)]
mgii_clump_ids = mgii_clump_ids[~np.isin(mgii_clump_ids, disk_clump_ids)]

hf = h5py.File("/Users/ctrapp/Documents/foggie_analysis/clump_project/clump_classifications/Tempest_DD0967_nref11c_nref9f_HVCs.h5",'w')
hf.create_dataset("hvc_clump_ids",data=hvc_leaf_ids)
hf.create_dataset("lvc_clump_ids",data=lvc_leaf_ids)
hf.create_dataset("ivc_clump_ids",data=ivc_leaf_ids)
hf.create_dataset("ovi_clump_ids",data=ivc_leaf_ids)
hf.create_dataset("mgii_clump_ids",data=ivc_leaf_ids)
hf.close()

for leaf_id in lvc_leaf_ids[0]:
    print("For Leaf:",leaf_id)
    print_clump_stats(clump_mass[leaf_id],clump_radius[leaf_id],clump_rvel[leaf_id],clump_vdoppler[leaf_id],clump_h_p0[leaf_id],clump_o_p6[leaf_id],clump_mg_p2[leaf_id])


In [ ]:
#Create the UCGs for each parameter
#This step takes a while, but is needed only for selecting individual clumps with napari
import trident
trident.add_ion_fields(ds, ions=['O VI','Mg II'])

print("Creating simple ucg (may take a while)...")
visualization_field = ('gas','Mg_p1_number_density')
#visualization_field = ('gas','density')
ucg_list = create_simple_ucg(ds, refine_box, [visualization_field,('gas','leaf_id')], 11, split_method=["copy","copy"],merge_method = ["max","max"])

In [ ]:
#Make 3-d projections of clumps of various criteria for selection of individual clumps
#In order to use this, toggle which ever field you wish to target (e.g. Mg II Clumps) and make sure that is your selected field
#Then, move your cursor over a clump of interest and get the clump id.
#This clump id can then be used to identify the clump for further analysis/tracer fluid tracking

import napari
datacubes=[]
ucg_list[0][ucg_list[0]==0] = np.min(ucg_list[0][ucg_list[0]>0])/10.
#ucg_list_ions[0][ucg_list_ions[0]==0] = np.min(ucg_list_ions[0][ucg_list_ions[0]>0])/10.
#ucg_list_ions[1][ucg_list_ions[1]==0] = np.min(ucg_list_ions[1][ucg_list_ions[1]>0])/10.

mg_ii_ids = np.copy(ucg_list[1])
o_vi_ids = np.copy(ucg_list[1])

mg_ii_ids[~np.isin(mg_ii_ids , mgii_clump_ids)] = -1
o_vi_ids[~np.isin(o_vi_ids , ovi_clump_ids)] = -1

viewer = napari.Viewer(ndisplay=3)
img_layer = viewer.add_image(np.log10(ucg_list[0]), colormap='inferno', name='Mg II', rendering='mip',depiction='volume',gamma=0.7)

viewer.add_image(np.round(ucg_list[1]).astype(int), colormap='inferno', name='Clump id', rendering='mip',depiction='volume',gamma=0.7)
viewer.add_image(np.round(mg_ii_ids).astype(int), colormap='green', name='Mg II Ids', rendering='mip',depiction='volume',gamma=0.2,opacity=0.6)
viewer.add_image(np.round(o_vi_ids).astype(int), colormap='cyan', name='O VI Ids', rendering='mip',depiction='volume',gamma=0.2,opacity=0.6)

viewer.camera.zoom=(2.5)



napari.run()